# Has the language has changed after the hype?

In [1]:
%%capture
from datetime import datetime
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
plt.rcParams.update({"font.size": 12})
%matplotlib inline

In [2]:
wsb_ = pd.read_csv('data/WSB_data_with_emb.csv',sep=';', usecols = ['created_utc','author'], low_memory = True)
wsb_.head(2)

,created_utc,author
0,2021-10-19 17:58:57,VendiVendiVendi
1,2021-10-19 17:57:49,Far_Bass_7284


In [3]:
stocks = pd.read_csv('stocks/data/stocks_submissions_all.csv',sep=';', usecols = ['created_utc','author'], low_memory = True)
stocks.head(1)

,created_utc,author
0,2021-11-26 13:19:32,NovelAnteater


In [4]:
investing = pd.read_csv('investing/data/investing_submissions_all.csv',sep=';', usecols = ['created_utc','author'], low_memory = True)
investing.head(2)

,created_utc,author
0,2021-11-26 15:17:50,plszvlko
1,2021-11-26 14:43:13,eopif


In [5]:
#create pre and post datasets
# extract year from utc
#wsb['created_utc'] = pd.to_datetime(wsb['created_utc'], unit='s')
wsb_['year'] = pd.DatetimeIndex(wsb_['created_utc']).year
stocks['year'] = pd.DatetimeIndex(stocks['created_utc']).year


In [6]:
wsb_.head(2)

,created_utc,author,year
0,2021-10-19 17:58:57,VendiVendiVendi,2021
1,2021-10-19 17:57:49,Far_Bass_7284,2021


In [7]:
investing = investing.drop(investing[investing['created_utc'].str.contains('cryptoservices')].index)

In [8]:
#investing['created_utc'] = investing['created_utc'].apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()
investing['year'] = pd.DatetimeIndex(investing['created_utc']).year

In [9]:
#count total users
print("Unique users:(wsb)",len(wsb_['author'].unique()))
print("Unique users:(stocks)",len(stocks['author'].unique()))
print("Unique users:(investing)",len(investing['author'].unique()))

Unique users:(wsb) 396127
Unique users:(stocks) 84609
Unique users:(investing) 73951


In [10]:
wsb_users = wsb_['author'].unique()
stocks_users = stocks['author'].unique()
investing_users = investing['author'].unique()

In [54]:
## calculate Jaccard Similarity, and Overlap Coeffient between communities 
def calculate_userbase_similarity(list1,list2):
    common_users = len(set(list1) & set(list2))
    print("Absolute common users between two:",common_users)
    js=(
        abs(common_users)
        /
        (abs(
            (len(set(list1))+ len(set(list2)))
             -abs(common_users)
            )
        )
        )*100
    print("Jaccard similarity:",js)
    print("Overlap co-efficient:",(common_users/min(len(set(list1)),len(set(list2))))*100)

In [12]:
calculate_userbase_similarity(wsb_users,stocks_users)

Absolute common users between two: 21271
Jaccard similarity: 4.6295147617337555
Overlap co-efficient: 25.14035149924949


In [13]:
calculate_userbase_similarity(wsb_users,investing_users)

Absolute common users between two: 16413
Jaccard similarity: 3.6178678099478687
Overlap co-efficient: 22.194426038863572


In [14]:
calculate_userbase_similarity(stocks_users,investing_users)

Absolute common users between two: 13964
Jaccard similarity: 9.657251929513956
Overlap co-efficient: 18.88277372854999


In [15]:
## Group by year
def create_dataset_groupby_author(df):
    df_author_posts= df.groupby(['year']).agg({'created_utc': 'count'})
    df_author_posts['authors']= df.groupby(['year']).agg({'author': ' '.join})     
    df_author_posts['count'] =df_author_posts['created_utc']
    df_author_posts['authors']=df_author_posts['authors'].apply(lambda x: x.split())
    return df_author_posts

In [16]:
#count users by year
wsb_by_year=create_dataset_groupby_author(wsb_)
stocks_by_year=create_dataset_groupby_author(stocks)
investing_by_year=create_dataset_groupby_author(investing)
# list of authors by year


In [17]:
wsb_by_year.head()

,created_utc,authors,count
year,,,
2018,80,"[dannyphantom567, reizeto, dmdmoses46, tinmast...",80
2019,43735,"[i2t4fun, actuallyhim, dbosspec, Speedm4ster, ...",43735
2020,174649,"[Watertuner_51, FatCatBoomerBanker, sam4gh, WA...",174649
2021,643288,"[VendiVendiVendi, Far_Bass_7284, HistoricalAni...",643288


In [18]:
stocks_by_year.head()

,created_utc,authors,count
year,,,
2018,62,"[FatexP, Luxbu, GreenBull7, RichTVLive, uracow...",62
2019,11133,"[ben-weezy, coolcomfort123, BostonBorn00, jynx...",11133
2020,57723,"[ttolam, DisruptiveTechn, williamsjohnwilliams...",57723
2021,85588,"[NovelAnteater, VonDerBerg, VonDerBerg, VonDer...",85588


In [19]:
investing_by_year.head()

,created_utc,authors,count
year,,,
2017,63,"[ChrisDurst, lookatthisfuckindoge, [deleted], ...",63
2018,14929,"[KevinMKZ, chriscasemart, baccigaloopa, Wza99,...",14929
2019,15664,"[101919chunkawy, BigBonyBaloney, GaniB, Invest...",15664
2020,44195,"[rawrtherapybackup, Dhamedd, InLoveWithMyDick,...",44195
2021,52917,"[plszvlko, eopif, VonDerBerg, AccountForFinanc...",52917


In [45]:
lst = ['101919chunkawy', 'BigBonyBaloney', 'GaniB','BigBonyBaloney']
set(lst)

{'101919chunkawy', 'BigBonyBaloney', 'GaniB'}

In [44]:
wsb_by_year[wsb_by_year.index==2020]['authors'].tolist()[0]

AttributeError: 'list' object has no attribute 'unique'

In [50]:
len(
    set(wsb_by_year[wsb_by_year.index==2020]['authors'].tolist()[0]) -                                                    
    set(wsb_by_year[wsb_by_year.index==2019]['authors'].tolist()[0])                        
   )

68677

In [27]:
len( 
                            set(wsb_by_year[wsb_by_year.index==2020]['authors'].tolist()[0]) &
                            set(wsb_by_year[wsb_by_year.index==2019]['authors'].tolist()[0])
                        )

5705

In [23]:
set({1,2,3,4,5} ) - set({1,2,3})

{4, 5}

In [20]:
 for year in [2019,2020,2021]:
        print("Year:",year)
        common_users = len((set(wsb_by_year[wsb_by_year.index==year]['authors'].tolist()[0]) & 
                            set(stocks_by_year[stocks_by_year.index==year]['authors'].tolist()[0])) 
                           & set(investing_by_year[investing_by_year.index==year]['authors'].tolist()[0]))
        print("Absolute common users between tree:",common_users)
        

Year: 2019
Absolute common users between tree: 288
Year: 2020
Absolute common users between tree: 1596
Year: 2021
Absolute common users between tree: 2742


In [52]:
def userbase_sim_by_year(df1,df2):
    for year in [2019,2020,2021]:
        print("Year:",year)
        if year == 2019:
            calculate_userbase_similarity(df1[df1.index==year]['authors'].tolist()[0],df2[df2.index==year]['authors'].tolist()[0])
        elif year == 2020:
            df1_list =list(set(df1[df1.index==2020]['authors'].tolist()[0]) -                                             
                        set(df1[df1.index==2019]['authors'].tolist()[0]) )
            df2_list =list(set(df2[df2.index==2020]['authors'].tolist()[0]) -                                             
                        set(df2[df2.index==2019]['authors'].tolist()[0]) )
            calculate_userbase_similarity(df1_list,df2_list)
        else:
            df1_list =list(set(df1[df1.index==2021]['authors'].tolist()[0]) -                                             
                          (set(df1[df1.index==2020]['authors'].tolist()[0]) - set(df1[df1.index==2019]['authors'].tolist()[0])) )
            df2_list =list(set(df2[df2.index==2021]['authors'].tolist()[0]) -                                             
                        (set(df2[df2.index==2020]['authors'].tolist()[0]) - set(df2[df2.index==2019]['authors'].tolist()[0])) )
            calculate_userbase_similarity(df1_list,df2_list)
                


In [55]:
userbase_sim_by_year(wsb_by_year,stocks_by_year)

Year: 2019
Absolute common users between two: 939
Jaccard similarity: 3.666679682923972
Overlap co-efficient: 14.287888009738284
Year: 2020
Absolute common users between two: 4532
Jaccard similarity: 4.912950155020272
Overlap co-efficient: 16.12753994519768
Year: 2021
Absolute common users between two: 10949
Jaccard similarity: 3.1268205755017644
Overlap co-efficient: 21.747075297435796


In [86]:
userbase_sim_by_year(wsb_by_year,stocks_by_year) #old incorrect

Year: 2019
Absolute common users between two: 939
Jaccard similarity: 0.017411782158022585
Overlap co-efficient: 0.08434384263001886
Year: 2020
Absolute common users between two: 5069
Jaccard similarity: 0.02230062955614312
Overlap co-efficient: 0.08781594858202103
Year: 2021
Absolute common users between two: 12986
Jaccard similarity: 0.01813965832739667
Overlap co-efficient: 0.1517268775996635


In [56]:
userbase_sim_by_year(wsb_by_year,investing_by_year) 

Year: 2019
Absolute common users between two: 1124
Jaccard similarity: 3.9202008928571432
Overlap co-efficient: 11.44602851323829
Year: 2020
Absolute common users between two: 4319
Jaccard similarity: 4.798942210469005
Overlap co-efficient: 16.844116844116844
Year: 2021
Absolute common users between two: 6169
Jaccard similarity: 1.8348869443139029
Overlap co-efficient: 19.516593375304502


In [87]:
userbase_sim_by_year(wsb_by_year,investing_by_year) #old incorrect

Year: 2019
Absolute common users between two: 1124
Jaccard similarity: 0.01928785928785929
Overlap co-efficient: 0.07175689479060265
Year: 2020
Absolute common users between two: 5086
Jaccard similarity: 0.023793261538749427
Overlap co-efficient: 0.11508089150356375
Year: 2021
Absolute common users between two: 7519
Jaccard similarity: 0.010917892914913328
Overlap co-efficient: 0.1420904435247652


In [57]:
userbase_sim_by_year(stocks_by_year,investing_by_year)

Year: 2019
Absolute common users between two: 946
Jaccard similarity: 6.124562993655315
Overlap co-efficient: 14.39440048691418
Year: 2020
Absolute common users between two: 4340
Jaccard similarity: 8.785069430387434
Overlap co-efficient: 16.926016926016928
Year: 2021
Absolute common users between two: 6203
Jaccard similarity: 8.188454582656792
Overlap co-efficient: 19.62415767661109


In [88]:
userbase_sim_by_year(stocks_by_year,investing_by_year) #old incorrect

Year: 2019
Absolute common users between two: 946
Jaccard similarity: 0.03659432903949557
Overlap co-efficient: 0.08497260397017875
Year: 2020
Absolute common users between two: 4846
Jaccard similarity: 0.0499217075984836
Overlap co-efficient: 0.10965041294264057
Year: 2021
Absolute common users between two: 7289
Jaccard similarity: 0.05554962809413486
Overlap co-efficient: 0.1377440142109341


In [ ]:
# users from stocks and investing were not active in WSB in 2019,2020, but started posting in 2021?

In [97]:
print(len(set(stocks_by_year[stocks_by_year.index==2019]['authors'].tolist()[0]) - set(wsb_by_year[wsb_by_year.index==2019]['authors'].tolist()[0])))

5633


In [102]:
stocks_usrs_not_active_in_wsb_2019 = set(stocks_by_year[stocks_by_year.index==2019]['authors'].tolist()[0]) - set(wsb_by_year[wsb_by_year.index==2019]['authors'].tolist()[0])
print(len(stocks_usrs_not_active_in_wsb_2019))

5633


In [112]:
stocks2019_usrs_ntactive_in_wsb_2020 = set(stocks_usrs_not_active_in_wsb_2019) - set(wsb_by_year[wsb_by_year.index==2020]['authors'].tolist()[0])
print(len(stocks2019_usrs_ntactive_in_wsb_2020))

5168


In [113]:
stocks2019_usrs_ntactive_in_wsb_2021 = set(stocks_usrs_not_active_in_wsb_2019) - set(wsb_by_year[wsb_by_year.index==2021]['authors'].tolist()[0])
print(len(stocks2019_usrs_ntactive_in_wsb_2021))

5189


In [110]:
stocks2019_usrs_active_in_wsb_2020 = set(stocks_usrs_not_active_in_wsb_2019) & set(wsb_by_year[wsb_by_year.index==2020]['authors'].tolist()[0])
print(len(stocks2019_usrs_active_in_wsb_2020))

465


In [103]:
stocks_usrs_not_active_in_wsb_2020 = set(stocks_by_year[stocks_by_year.index==2020]['authors'].tolist()[0]) - set(wsb_by_year[wsb_by_year.index==2020]['authors'].tolist()[0])
print(len(stocks_usrs_not_active_in_wsb_2020))

24040


In [114]:
stocks2020_usrs_not_active_in_wsb_2021 = set(stocks_by_year[stocks_by_year.index==2020]['authors'].tolist()[0]) - set(wsb_by_year[wsb_by_year.index==2021]['authors'].tolist()[0])
print(len(stocks2020_usrs_not_active_in_wsb_2021))

24655


In [104]:
stocks2019_usrs_active_in_wsb_2021 = set(stocks_usrs_not_active_in_wsb_2019) & set(wsb_by_year[wsb_by_year.index==2021]['authors'].tolist()[0])
print(len(stocks2019_usrs_active_in_wsb_2021))

444


In [105]:
stocks2020_usrs_active_in_wsb_2021 = set(stocks_usrs_not_active_in_wsb_2020) & set(wsb_by_year[wsb_by_year.index==2021]['authors'].tolist()[0])
print(len(stocks2020_usrs_active_in_wsb_2021))

2584


In [ ]:
#more than 11% of users from stocks who were inactve in WSB in year 2020 posted in WSB in 2021

In [108]:
investing_usrs_not_active_in_wsb_2019 = set(investing_by_year[investing_by_year.index==2019]['authors'].tolist()[0]) - set(wsb_by_year[wsb_by_year.index==2019]['authors'].tolist()[0])
print(len(investing_usrs_not_active_in_wsb_2019))
investing_usrs_not_active_in_wsb_2020 = set(investing_by_year[investing_by_year.index==2020]['authors'].tolist()[0]) - set(wsb_by_year[wsb_by_year.index==2020]['authors'].tolist()[0])
print(len(investing_usrs_not_active_in_wsb_2020))
investing2019_usrs_active_in_wsb_2021 = set(investing_usrs_not_active_in_wsb_2019) & set(wsb_by_year[wsb_by_year.index==2021]['authors'].tolist()[0])
print(len(investing2019_usrs_active_in_wsb_2021))
investing2020_usrs_active_in_wsb_2021 = set(investing_usrs_not_active_in_wsb_2020) & set(wsb_by_year[wsb_by_year.index==2021]['authors'].tolist()[0])
print(len(investing2020_usrs_active_in_wsb_2021))

8696
22028
528
1808


In [109]:
set({1,2,3})-set({15,2,3,3,3,35,5})

{1}

In [92]:
def userbase_migration_by_year(df1,df2):
    for year in [2019,2020,2021]:
        print("Year:",year)
        for year2 in [2019,2020,2021]:
            print("Year2:",year2)
            #calculate_userbase_similarity(df1[df1.index==year]['authors'].tolist()[0],df2[df2.index==year2]['authors'].tolist()[0])
            calculate_userbase_similarity(df1[df1.index==year]['authors'].tolist()[0],df2[df2.index==year2]['authors'].tolist()[0])

In [93]:
userbase_migration_by_year(wsb_by_year,stocks_by_year)

Year: 2019
Year2: 2019
Absolute common users between two: 939
Jaccard similarity: 0.017411782158022585
Overlap co-efficient: 0.08434384263001886
Year2: 2020
Absolute common users between two: 555
Jaccard similarity: 0.005500332002021744
Overlap co-efficient: 0.012690065165199497
Year2: 2021
Absolute common users between two: 383
Jaccard similarity: 0.0029703738172793545
Overlap co-efficient: 0.008757288213101636
Year: 2020
Year2: 2019
Absolute common users between two: 834
Jaccard similarity: 0.004509375608279084
Overlap co-efficient: 0.07491242252762059
Year2: 2020
Absolute common users between two: 5069
Jaccard similarity: 0.02230062955614312
Overlap co-efficient: 0.08781594858202103
Year2: 2021
Absolute common users between two: 2353
Jaccard similarity: 0.009124257418063936
Overlap co-efficient: 0.027492171799785015
Year: 2021
Year2: 2019
Absolute common users between two: 671
Jaccard similarity: 0.0010263862332695985
Overlap co-efficient: 0.06027126560675469
Year2: 2020
Absolute co

In [95]:
userbase_migration_by_year(wsb_by_year,investing_by_year)

Year: 2019
Year2: 2019
Absolute common users between two: 1124
Jaccard similarity: 0.01928785928785929
Overlap co-efficient: 0.07175689479060265
Year2: 2020
Absolute common users between two: 861
Jaccard similarity: 0.009888708954966751
Overlap co-efficient: 0.01968674974276895
Year2: 2021
Absolute common users between two: 339
Jaccard similarity: 0.003519774069959403
Overlap co-efficient: 0.007751228992797531
Year: 2020
Year2: 2019
Absolute common users between two: 977
Jaccard similarity: 0.00516013858959733
Overlap co-efficient: 0.062372318692543414
Year2: 2020
Absolute common users between two: 5086
Jaccard similarity: 0.023793261538749427
Overlap co-efficient: 0.11508089150356375
Year2: 2021
Absolute common users between two: 1827
Jaccard similarity: 0.008093417619463186
Overlap co-efficient: 0.034525766766823514
Year: 2021
Year2: 2019
Absolute common users between two: 753
Jaccard similarity: 0.0011440309085853973
Overlap co-efficient: 0.04807201225740552
Year2: 2020
Absolute com

In [96]:
userbase_migration_by_year(stocks_by_year,investing_by_year)

Year: 2019
Year2: 2019
Absolute common users between two: 946
Jaccard similarity: 0.03659432903949557
Overlap co-efficient: 0.08497260397017875
Year2: 2020
Absolute common users between two: 587
Jaccard similarity: 0.01072322390895307
Overlap co-efficient: 0.052726129524836074
Year2: 2021
Absolute common users between two: 248
Jaccard similarity: 0.003887025485094511
Overlap co-efficient: 0.022276116051378785
Year: 2020
Year2: 2019
Absolute common users between two: 604
Jaccard similarity: 0.008298641166206395
Overlap co-efficient: 0.038559754851889685
Year2: 2020
Absolute common users between two: 4846
Jaccard similarity: 0.0499217075984836
Overlap co-efficient: 0.10965041294264057
Year2: 2021
Absolute common users between two: 1445
Jaccard similarity: 0.013233206648656074
Overlap co-efficient: 0.027306914602112743
Year: 2021
Year2: 2019
Absolute common users between two: 359
Jaccard similarity: 0.003558225050300814
Overlap co-efficient: 0.02291879468845761
Year2: 2020
Absolute common

## Put it on temporal scale / Time Period -2018- 2021 - how have the features changes/evolved over time?

### pre vs post n-grams

In [13]:
import re
def count_ngrams(ngram_column):
    trigrams_title = []
    for lst in ngram_column.tolist():  
          for trigram in lst.split(','):    
            trigram=re.sub('[[]', '', trigram)
            trigram=re.sub('[]]', '', trigram)
            if trigram != '':      
                trigrams_title.append(trigram.lower())
    count_freq_trigrams = {}
    for item in trigrams_title:
        item = item.strip()
        if item in count_freq_trigrams:
            count_freq_trigrams[item] +=1
        else:
            count_freq_trigrams[item] = 1
    return sorted(count_freq_trigrams.items(), key=lambda item: item[1], reverse = True)

### bigrams

#### WSB

In [14]:
print(count_ngrams(wsb_pre['bigram'])[:10])

C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Possible nested set at position 1
  


[('🚀🚀', 7786), ('🌈🐻', 2208), ('gonna', 1497), ('$tsla', 1338), ('000', 1172), ('stock market', 1151), ('wsbvotebot log', 1077), ('02 2020', 888), ('spy puts', 886), ('wall street', 695)]


In [15]:
list(list(zip(*count_ngrams(wsb_pre['bigram'])[:15]))[0])

['🚀🚀',
 '🌈🐻',
 'gonna',
 '$tsla',
 '000',
 'stock market',
 'wsbvotebot log',
 '02 2020',
 'spy puts',
 'wall street',
 'long $',
 'loss porn',
 'looks like',
 'free money',
 '$spy']

In [16]:
print(count_ngrams(wsb_post['bigram'])[:10])

[('🚀🚀', 176355), ('$gme', 13915), ('💎🙌', 13516), ('💎💎', 12326), ('🚀 🚀', 6820), ('🦍🦍', 6341), ('$amc', 5772), ('diamond hands', 5345), ('moon 🚀', 4986), ('short squeeze', 4800)]


In [17]:
list(list(zip(*count_ngrams(wsb_post['bigram'])[:15]))[0])

['🚀🚀',
 '$gme',
 '💎🙌',
 '💎💎',
 '🚀 🚀',
 '🦍🦍',
 '$amc',
 'diamond hands',
 'moon 🚀',
 'short squeeze',
 '🚀💎',
 'wall street',
 'hedge funds',
 '$bb',
 'gonna']

In [43]:
## unique between pre and post from TOP 1000
list(set(list(list(zip(*count_ngrams(wsb_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(wsb_pre['bigram'])[:1000]))[0])))[:15]

['bb yolo',
 'ted cruz',
 '🏾💎',
 'holding 🚀',
 'apes 🚀',
 '🦍🦍',
 'trading apps',
 'short float',
 '👐🏻',
 '💎 💎',
 'gme tendies',
 'line boys',
 'borrow fee',
 'gme right',
 'nt let']

In [24]:
len(list(set(list(list(zip(*count_ngrams(wsb_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(wsb_pre['bigram'])[:1000]))[0]))))

606

In [60]:
uniq_post =list(set(list(list(zip(*count_ngrams(wsb_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(wsb_pre['bigram'])[:1000]))[0])))[:606]

In [61]:
[item for item in count_ngrams(wsb_post['bigram'])[:1000] if item[0]  in uniq_post][:20]

[('🦍🦍', 6341),
 ('$amc', 5772),
 ('🚀💎', 4382),
 ('🙌💎', 3518),
 ('💎🚀', 3299),
 ('🙌🏻', 2350),
 ('$wish', 2323),
 ('🙌🏼', 1973),
 ('💎 🙌', 1872),
 ('hold hold', 1829),
 ('🦍🚀', 1804),
 ('$clov', 1681),
 ('🙌🚀', 1570),
 ('🙌🙌', 1511),
 ('💎🤚', 1489),
 ('gme $', 1466),
 ('financial advice', 1417),
 ('melvin capital', 1359),
 ('fellow apes', 1349),
 ('🏻💎', 1184)]

#### Stocks

In [25]:
print(count_ngrams(stocks_pre['bigram'])[:10])

[('stock market', 1053), ('long term', 961), ('$$', 460), ('daily discussion', 368), ('stocks daily', 367), ('000', 278), ('s&amp;p 500', 272), ('day trading', 268), ('stock price', 232), ('good time', 229)]


In [26]:
list(list(zip(*count_ngrams(stocks_pre['bigram'])[:11]))[0])

['stock market',
 'long term',
 '$$',
 'daily discussion',
 'stocks daily',
 '000',
 's&amp;p 500',
 'day trading',
 'stock price',
 'good time',
 'guys think']

In [27]:
print(count_ngrams(stocks_post['bigram'])[:10])

[('🚀🚀', 1391), ('stock market', 1038), ('long term', 950), ('short squeeze', 470), ('stocks daily', 417), ('need help', 349), ('meme stocks', 337), ('$gme', 283), ('000', 281), ('stock price', 263)]


In [28]:
list(list(zip(*count_ngrams(stocks_post['bigram'])[:10]))[0])

['🚀🚀',
 'stock market',
 'long term',
 'short squeeze',
 'stocks daily',
 'need help',
 'meme stocks',
 '$gme',
 '000',
 'stock price']

In [46]:
list(set(list(list(zip(*count_ngrams(stocks_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(stocks_pre['bigram'])[:1000]))[0])))[:15]

['black swan',
 '🦍🦍',
 '🚀🌙',
 '$nok',
 '$f',
 'young investors',
 'short float',
 'short volume',
 '💥💥',
 'bullish defi',
 'debt ceiling',
 'securities fraud',
 '3d printing',
 'quarter 2021',
 'shiba inu']

In [30]:
len(list(set(list(list(zip(*count_ngrams(stocks_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(stocks_pre['bigram'])[:1000]))[0]))))

395

In [57]:
uniq_post =list(set(list(list(zip(*count_ngrams(stocks_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(stocks_pre['bigram'])[:1000]))[0])))[:395]

In [58]:
[item for item in count_ngrams(stocks_post['bigram'])[:1000] if item[0]  in uniq_post][:20]

[('$gme', 283),
 ('daily thread', 198),
 ('$amc', 134),
 ('overlooked stocks', 93),
 ('stocks discuss', 93),
 ('discuss overlooked', 91),
 ('amc stock', 88),
 ('doge coin', 85),
 ('🔥🔥', 83),
 ('buy gme', 80),
 ('gift card', 75),
 ('💎💎', 69),
 ('gme short', 69),
 ('$nok', 67),
 ('market stock', 65),
 ('gme stock', 64),
 ('stock movers', 61),
 ('stocks thursday', 58),
 ('stocks wednesday', 58),
 ('stocks friday', 57)]

####  Investing

In [33]:
print(count_ngrams(investing_pre['bigram'])[:10])

[('stock market', 1121), ('long term', 896), ('roth ira', 859), ('daily advice', 822), ('advice thread', 820), ('000', 550), ('s&amp;p 500', 545), ('index funds', 483), ('basic help', 443), ('advice questions', 443)]


In [34]:
list(list(zip(*count_ngrams(investing_pre['bigram'])[:10]))[0])

['stock market',
 'long term',
 'roth ira',
 'daily advice',
 'advice thread',
 '000',
 's&amp;p 500',
 'index funds',
 'basic help',
 'advice questions']

In [35]:
print(count_ngrams(investing_post['bigram'])[:10])

[('long term', 723), ('roth ira', 667), ('stock market', 584), ('🚀🚀', 548), ('daily advice', 302), ('basic help', 302), ('advice thread', 301), ('advice questions', 301), ('000', 298), ('index funds', 297)]


In [36]:
list(list(zip(*count_ngrams(investing_post['bigram'])[:10]))[0])

['long term',
 'roth ira',
 'stock market',
 '🚀🚀',
 'daily advice',
 'basic help',
 'advice thread',
 'advice questions',
 '000',
 'index funds']

In [45]:
list(set(list(list(zip(*count_ngrams(investing_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(investing_pre['bigram'])[:1000]))[0])))[:15]

['stock day',
 'black swan',
 'fund performance',
 'greatly appreciated',
 'trading apps',
 '$nok',
 'principal solar',
 'earn free',
 'short float',
 'new bitcoin',
 'student coin',
 'rolls royce',
 'portfolio diversification',
 'invitation code',
 'etf recommendations']

In [53]:
uniq_post =list(set(list(list(zip(*count_ngrams(investing_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(investing_pre['bigram'])[:1000]))[0])))[:387]

In [56]:
[item for item in count_ngrams(investing_post['bigram'])[:1000] if item[0]  in uniq_post][:20]

[('daily general', 259),
 ('general discussion', 259),
 ('spitballin thread', 256),
 ('$gme', 150),
 ('🍀🍀', 70),
 ('👍🏾', 56),
 ('meme stocks', 52),
 ('🏾👍', 52),
 ('$amc', 49),
 ('🔥🔥', 47),
 ('cathie wood', 45),
 ('⠀⠀', 44),
 ('💰💰', 43),
 ('big thing', 39),
 ('| ant', 37),
 ('$bb', 37),
 ('gift card', 35),
 ('meme stock', 35),
 ('financial advice', 34),
 ('buy gme', 34)]

In [37]:
len(list(set(list(list(zip(*count_ngrams(investing_post['bigram'])[:1000]))[0])) - set(list(list(zip(*count_ngrams(investing_pre['bigram'])[:1000]))[0]))))

387

### Tri-grams

#### WSB

In [ ]:
print(count_ngrams(wsb_pre['trigram'])[:100])

In [ ]:
print(count_ngrams(wsb_post['trigram'])[:100])

#### Stocks

In [ ]:
print(count_ngrams(stocks_pre['trigram'])[:100])

In [ ]:
print(count_ngrams(stocks_post['trigram'])[:100])

#### Investing

In [ ]:
print(count_ngrams(investing_pre['trigram'])[:100])

In [ ]:
print(count_ngrams(investing_post['trigram'])[:100])

### Four-grams

#### WSB

In [ ]:
print(count_ngrams(wsb_pre['four-gram'])[:50])

In [ ]:
print(count_ngrams(wsb_post['four-gram'])[:50])

#### Stocks

In [ ]:
print(count_ngrams(stocks_pre['four-gram'])[:100])

In [ ]:
print(count_ngrams(stocks_post['four-gram'])[:100])

#### Investing

In [ ]:
print(count_ngrams(investing_pre['four-gram'])[:100])

In [ ]:
print(count_ngrams(investing_post['four-gram'])[:100])

### Five-grams

#### WSB

In [ ]:
print(count_ngrams(wsb_pre['five-gram'])[:50])

In [ ]:
print(count_ngrams(wsb_post['five-gram'])[:50])

#### Stocks

In [ ]:
print(count_ngrams(stocks_pre['five-gram'])[:50])

In [ ]:
print(count_ngrams(stocks_post['five-gram'])[:50])

#### Investing

In [ ]:
print(count_ngrams(investing_pre['five-gram'])[:50])

In [ ]:
print(count_ngrams(investing_post['five-gram'])[:50])

In [ ]:
## To do: Intersection between all sets to find common n-grams

## Plot n-grams [Daily, Weekly & Monthly]

In [2]:
# load datasets
stocks_pre = pd.read_csv('stocks/data/stocks_pre.csv',sep=';')
stocks_post = pd.read_csv('stocks/data/stocks_post.csv',sep=';')

In [3]:
stocks_pre.head(1)

,Unnamed: 0,created_utc,SegmentID,author,created_utc.1,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio,sbert_emb,bigram,trigram,four-gram,five-gram,year
0,85588,2020-12-31 23:58:19,85588,ttolam,2020-12-31 23:58:19,ko0zlv,NaN,27,1,1002204,I have a Roth IRA\n\nI am going to put $3500 i...,How should I spend my $3500,1.0,"[0.325336754322052, -0.6045418381690979, 0.058...",[$3500],[spend my $],[spend my $3500],[],2020


In [ ]:
#pre-post
wsb_popular_bigrams = ['👙👙','💜🤎','🚀🚀','🤎💜','🤬🤬']
wsb_popular_trigrams = ['low quality meme','trade vix options','🚀🚀🚀','💜🤎💜','🤎💜🤎','🤬🤬🤬']
wsb_popular_fourgrams = ['blocking a low quality','🚀🚀🚀🚀','💜🤎💜🤎','🤎💜🤎💜','🤬🤬🤬🤬']
wsb_popular_fivegrams = ['blocking a low quality meme','💜🤎💜🤎💜','🚀🚀🚀🚀🚀','🤎💜🤎💜🤎','🤬🤬🤬🤬🤬']

#pre
wsb_popular_bigrams = ['👙👙','🚀🚀','🤬🤬']
wsb_popular_trigrams = ['low quality meme','trade vix options','🚀🚀🚀','🤬🤬🤬']
wsb_popular_fourgrams = ['blocking a low quality','🚀🚀🚀🚀','🤬🤬🤬🤬']
wsb_popular_fivegrams = ['blocking a low quality meme','🚀🚀🚀🚀🚀','🤬🤬🤬🤬🤬']

#post
#pre
wsb_popular_bigrams = ['👙👙','💜🤎','🚀🚀','🤎💜']
wsb_popular_trigrams = ['🚀🚀🚀','💜🤎💜','🤎💜🤎']
wsb_popular_fourgrams = ['🚀🚀🚀🚀','💜🤎💜🤎','🤎💜🤎💜']
wsb_popular_fivegrams = ['💜🤎💜🤎💜','🚀🚀🚀🚀🚀','🤎💜🤎💜🤎']

In [3]:
def count_ngrams_(ngram_column,min_count):
    #print(ngram_column[0])
    #print(ngram_column[1])
    trigrams_title = []
    for lst in ngram_column[0]:  
          for trigram in lst.split(','):
            if trigram == '[]':
               continue 
            #print(type(trigram))
            trigram=re.sub('\\[', '', trigram)
            trigram=re.sub('\\]', '', trigram)           
            if trigram != ' ' or trigram != '':      
                trigrams_title.append(trigram.lower())
    count_freq_trigrams = {}
    for item in trigrams_title:
        item = item.strip()
        if item in count_freq_trigrams:
            count_freq_trigrams[item] +=1
        else:
            count_freq_trigrams[item] = 1
    count_freq_trigrams_ = {}
    for item in count_freq_trigrams:
        #if (count_freq_trigrams[item]/ngram_column[1]) >= min_count:
        count_freq_trigrams_[item] =count_freq_trigrams[item]/ngram_column[1]
   
    return sorted(count_freq_trigrams_.items(), key=lambda item: item[1], reverse = True)
    #return count_freq_trigrams

In [13]:
def count_ngrams_abs(ngram_column):
    #print(ngram_column[0])
    #print(ngram_column[1])
    trigrams_title = []
    for lst in ngram_column[0]:  
          for trigram in lst.split(','):
            if trigram == '[]':
               continue 
            #print(type(trigram))
            trigram=re.sub('\\[', '', trigram)
            trigram=re.sub('\\]', '', trigram)           
            if trigram != ' ' or trigram != '':      
                trigrams_title.append(trigram.lower())
    count_freq_trigrams = {}
    for item in trigrams_title:
        item = item.strip()
        if item in count_freq_trigrams:
            count_freq_trigrams[item] +=1
        else:
            count_freq_trigrams[item] = 1
    #for item in count_freq_trigrams:
    #    count_freq_trigrams[item] =abs(count_freq_trigrams[item]#/ngram_column[1])
    #print(sorted(count_freq_trigrams.items(), key=lambda item: item[1], reverse = True))
    return sorted(count_freq_trigrams.items(), key=lambda item: item[1], reverse = True)
    #return count_freq_trigrams

In [4]:
import os
import re
#import gc


def plot_term_frequency(df1,column_name,time_period,save_dir,min_count = 0.25):
    #create date,Week,Month column
   
    df1['date'] = pd.DatetimeIndex(df1['created_utc']).date 
    if time_period == 'month':       
        grp_format = '%m-%Y'
    elif time_period == 'year':       
        grp_format = '%Y'
    elif time_period == 'week':
        #df1[time_period] = pd.DatetimeIndex(df1['created_utc']).isocalendar().week       
        #df1['month'] = pd.DatetimeIndex(df1['created_utc']).month    %U
        grp_format = '%Y_%U'
    elif time_period == 'daily':       
        grp_format = '%d-%m-%Y'
    else:
      print('incorrect time period')
      return
    #Count term freqs  by time_period
    #group by Month, join()

    post_count = df1.groupby(pd.to_datetime(df1['created_utc']).dt.strftime(grp_format))[column_name].count().reset_index()
    #print(post_count[column_name].tolist())
    bigram = df1.groupby(pd.to_datetime(df1['created_utc']).dt.strftime(grp_format))[column_name].agg(list) 
    #bigram[['post_count']] = post_count[column_name]
    bigram_new=pd.DataFrame(bigram)
    bigram_new['post_count'] = post_count[column_name].tolist()
    #print(bigram_new)
    #return
    if 'norm' in save_dir:
         bigram_= bigram_new.apply(lambda x: count_ngrams_(x,min_count),axis=1)
    else:
         bigram_= bigram_new.apply(lambda x: count_ngrams_abs(x),axis=1)
    #return
    frame = { column_name: bigram_ }
    df_plot= pd.DataFrame(frame)

    df_plot[column_name+"_dict"] = df_plot.apply(lambda x: dict(x[column_name]), axis=1)  
    #print(df_plot)
    #return
    df_plot_= pd.json_normalize(df_plot[column_name+"_dict"])
    df_plot = df_plot.rename_axis('created_utc').reset_index()
    df_plot_['created_utc']= df_plot['created_utc']
    #print(df_plot_)
    df_plot_=df_plot_.set_index('created_utc')   
    df_plot_= df_plot_.fillna(0)
    if (time_period != 'week'):
        df_plot_.index = pd.to_datetime(df_plot_.index)
        df_plot_.sort_index(inplace=True)   
        
    #print(df_plot_)
    #return
    count =0    
    path1 = save_dir+"\\"+column_name
    path2 = save_dir+"\\"+column_name+"\\"+ time_period
    path3 =  save_dir+"\\"+column_name+"\\"+ time_period+"\\"+str(min_count)
    if not os.path.exists(path1):
        os.mkdir(path1)
      
    if not os.path.exists(path2):
        os.mkdir(path2)
    
    #print((df_plot_.sum(axis=0)))
    
    #df_plot_ = df_plot_.loc[:, (df_plot_.sum(axis=0) >= min_count)]
    #print(df_plot_.columns)
    #return
    #display(df_plot_.head())
    print(len(df_plot_))
    
    if len(df_plot_.columns) >0:
        if not os.path.exists(path3):
            os.mkdir(path3)
        for col in df_plot_.columns:
            val = (df_plot_[col].sum()/len(df_plot_[col])) 
            #print(val)
            if val >= min_count:
              #fig=''
              #print(df_plot_.columns[count])
              plt.figure(1)
              if (time_period == 'daily') or  (time_period == 'week'):            
                  fig, axs = plt.subplots(figsize = (30,10))
              else:
                  fig, axs = plt.subplots(figsize = (15,8))
              #print(df_plot_.columns[count])

              plt.xticks(rotation=45, ha='right')
              plt.title(df_plot_.columns[count].replace('$','\\$'), fontdict=None, loc='center')
              plt.plot(df_plot_.index,df_plot_[col])    
              os.chmod(path2, 0o400)

              try:    
                      plt.ioff()
                      plt.savefig(path3+"\\"+column_name+'_'+df_plot_.columns[count] +'.png')
                      plt.close(1)
                      plt.ioff()
                      count = count + 1
              except OSError:

                print("can't save:"+df_plot_.columns[count])
                count = count + 1
              

    del [[df1,df_plot,df_plot_]]
    return count
    #gc.collect()
    #df1=pd.DataFrame()
    #df_plot=pd.DataFrame()
    #df_plot_=pd.DataFrame()
      

In [7]:
# load datasets
#investing_pre = pd.read_csv('investing/data/investing_pre.csv',sep=';')
#investing_post = pd.read_csv('investing/data/investing_post.csv',sep=';')
investing = pd.read_csv('investing/data/investing_submissions_all.csv',sep=';')
investing = investing.drop(investing[investing['created_utc'].str.contains('cryptoservices')].index)


C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (1,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
investing_pre = investing_pre.drop(investing_pre[investing_pre['created_utc'].str.contains('cryptoservices')].index)
investing_post = investing_post.drop(investing_post[investing_post['created_utc'].str.contains('cryptoservices')].index)

In [6]:
# load datasets
wsb_pre = pd.read_csv('data/wsb_pre.csv',sep=';')
wsb_post = pd.read_csv('data/wsb_post.csv',sep=';')

In [7]:
##%%capture
ngrams =['trigram']#,'trigram','four-gram','five-gram']#'bigram','trigram',
time_periods = ['daily']#,'week','month','year']
dfs=[wsb_]#wsb_pre,wsb_post,
paths =["\\wsb\\plots\\pre-post\\norm"]#,"\\wsb\\plots\\post\\norm","\\wsb\\plots\\pre-post\\norm"]
min_counts = [0.001]#,0.25,0.50,1]
for df,path in zip(dfs,paths):
    for ngram in ngrams:
        for time_p in time_periods:
            for min_count in min_counts:
                print(ngram,time_p,min_count,":")
                print(plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count))
                #plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count)      

trigram daily 0.001 :


MemoryError: Unable to allocate 7.87 KiB for an array with shape (1007,) and data type uint64

In [8]:
stocks = pd.read_csv('stocks/data/stocks_submissions_all.csv',sep=';')

In [ ]:
##%%capture
ngrams =['bigram']#,'trigram','four-gram','five-gram']#'bigram','trigram',
time_periods = ['daily']#,'week','month','year']
dfs=[stocks]#[stocks_pre,stocks_post,
paths =["\\stocks\\plots\\pre-post\\norm"]#["\\stocks\\plots\\pre\\norm","\\stocks\\plots\\post\\norm",
min_counts = [0.05]
for df,path in zip(dfs,paths):
    for ngram in ngrams:
        for time_p in time_periods:
            for min_count in min_counts:
                print(ngram,time_p,min_count,":")
                print(plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count))
                #plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count)      

In [ ]:
%%capture
ngrams =['bigram','trigram','four-gram','five-gram']#'bigram',
time_periods = ['daily','week','month','year']#'daily',
dfs=[wsb_post]
paths =["\\wsb\\plots\\post\\all"]# ["\\stocks\\plots\\all","\\investing\\plots\\all","\\wsb\\plots\\all"]
min_counts = [5000]
for df,path in zip(dfs,paths):
    for ngram in ngrams:
        for time_p in time_periods:
            for min_count in min_counts:
                print(ngram,time_p,min_count,":")
                print(plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count))
                #plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count)   

In [ ]:
ngrams =['bigram','trigram','four-gram','five-gram']#'bigram','trigram',
time_periods = ['daily','week','month']
dfs=[investing]#investing_pre,investing_post,
paths =["\\investing\\plots\\pre-post\\norm"]# ["\\stocks\\plots\\all","\\investing\\plots\\all","\\wsb\\plots\\all"]"\\investing\\plots\\pre\\all","\\investing\\plots\\post\\all",
min_counts = [0.05]
for df,path in zip(dfs,paths):
    for ngram in ngrams:
        for time_p in time_periods:
            for min_count in min_counts:
                print(ngram,time_p,min_count," :")
                print(plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count))
                #plot_term_frequency(df,ngram,time_p,os.getcwd()+path,min_count)   

In [7]:
plt.close('all')

## Topic clusters, Modeling


In [ ]:
# load datasets

#wsb_post.to_csv('data/WSB_post.csv',sep=';')
stocks_pre = pd.read_csv('/content/drive/MyDrive/stocks/stocks_pre.csv',sep=';')
stocks_post = pd.read_csv('/content/drive/MyDrive/stocks/stocks_post.csv',sep=';')


In [ ]:
investing_pre = pd.read_csv('/content/drive/MyDrive/investing/investing_pre.csv',sep=';')
investing_post = pd.read_csv('/content/drive/MyDrive/investing/investing_post.csv',sep=';')

In [ ]:
#wsb_pre = pd.read_csv('/content/drive/MyDrive/WSB/WSB_pre.csv',sep=';')
wsb_post = pd.read_csv('/content/drive/MyDrive/WSB/WSB_post.csv',sep=';')

In [ ]:
%%capture
pip install bertopic

In [ ]:
pip install --upgrade tbb

In [ ]:
from bertopic import BERTopic
import re

In [ ]:
def bert_topic_modeling(df,n):
    
    df.index = pd.DatetimeIndex(df.created_utc)
    df_title = df["title"].apply(lambda title: re.sub(r"http\S+", "", title).lower(), 1)
    timestamps = df["created_utc"].to_list()
    titles = df_title.to_list()
    topics, _ = topic_model.fit_transform(titles[0:n])
    #freq = topic_model.get_topic_info(); freq.head(10)
    #topic_model.visualize_barchart(top_n_topics=3)
    return topics
    

In [ ]:
lst = [stocks_pre, stocks_post, investing_pre,investing_post]
del lst

### WSB

In [ ]:
len(wsb_pre['title'])

In [ ]:
#pre
%%capture
topic_model = BERTopic( verbose=True)
bert_topic_modeling(wsb_pre,99999)
#bert_topic_modeling(wsb_pre,len(wsb_pre['title']))

In [ ]:
topic_model.save("WSB_Pre_BERTopic_DTM_30")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english", min_df=10)
topic_model_ = BERTopic(vectorizer_model=vectorizer_model)

In [ ]:
t=BERTopic.load("WSB_Post_BERTopic_DTM_30")

In [ ]:
tops,_ = t.transform(wsb_post['title'].tolist()[0:99998])

In [ ]:
%%capture
topic_model = BERTopic( verbose=True)
bert_topic_modeling(wsb_post,99999)
#bert_topic_modeling(wsb_pre,len(wsb_pre['title']))

In [ ]:
topic_model.save("WSB_Post_BERTopic_DTM_30")

In [ ]:
from umap import UMAP
import matplotlib
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_20newsgroups

%matplotlib inline

# Create topic model
#docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))["data"]
#topic_model = BERTopic(embedding_model="paraphrase-MiniLM-L6-v2", verbose=True)
#topics, _ = topic_model.fit_transform(docs[0:99])
#df_title = wsb_post["title"].apply(lambda title: re.sub(r"http\S+", "", title).lower(), 1)
# Prepare data for plotting
embeddings = t._extract_embeddings(wsb_post['title'].tolist()[0:99998], method="document")
umap_model = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit(embeddings)
df = pd.DataFrame(umap_model.embedding_, columns=["x", "y"])
#topics, _ = topic_model.fit_transform(titles[0:99])
df["topic"] = tops

# Plot parameters
top_n = 10
fontsize = 12

# Slice data
to_plot = df.copy()
to_plot[df.topic >= top_n] = -1
outliers = to_plot.loc[to_plot.topic == -1]
non_outliers = to_plot.loc[to_plot.topic != -1]

# Visualize topics
cmap = matplotlib.colors.ListedColormap(['#FF5722', # Red
                                         '#03A9F4', # Blue
                                         '#4CAF50', # Green
                                         '#80CBC4', # FFEB3B
                                         '#673AB7', # Purple
                                         '#795548', # Brown
                                         '#E91E63', # Pink
                                         '#212121', # Black
                                         '#00BCD4', # Light Blue
                                         '#CDDC39', # Yellow/Red
                                         '#AED581', # Light Green
                                         '#FFE082', # Light Orange
                                         '#BCAAA4', # Light Brown
                                         '#B39DDB', # Light Purple
                                         '#F48FB1', # Light Pink
                                         ])

# Visualize outliers + inliers
fig, ax = plt.subplots(figsize=(15, 15))
scatter_outliers = ax.scatter(outliers['x'], outliers['y'], c="#E0E0E0", s=1, alpha=.3)
scatter = ax.scatter(non_outliers['x'], non_outliers['y'], c=non_outliers['topic'], s=1, alpha=.3, cmap=cmap)

# Add topic names to clusters
centroids = to_plot.groupby("topic").mean().reset_index().iloc[1:]
for row in centroids.iterrows():
    topic = int(row[1].topic)
    text = f"{topic}: " + "_".join([x[0] for x in t.get_topic(topic)[:3]])
    ax.text(row[1].x, row[1].y*1.01, text, fontsize=fontsize, horizontalalignment='center')

ax.text(0.99, 0.01, f"BERTopic - Top {top_n} topics", transform=ax.transAxes, horizontalalignment="right", color="black")
plt.xticks([], [])
plt.yticks([], [])
plt.show()

In [ ]:
df.head()

In [ ]:
 print(topic_model.get_topic)

In [ ]:
topic_model.visualize_barchart(top_n_topics=30)

### Stocks

In [ ]:
len(stocks_pre['title'])

In [ ]:
%%capture
topic_model = BERTopic( verbose=True)
#bert_topic_modeling(stocks_pre,50000)
bert_topic_modeling(stocks_pre,len(stocks_pre['title']))

In [ ]:
topic_model.save("Stocks_Pre_BERTopic_DTM_30")

In [ ]:
topic_model.visualize_barchart(top_n_topics=30)

In [ ]:
len(stocks_post['title'])

In [ ]:
%%capture
topic_model = BERTopic( verbose=True)
#bert_topic_modeling(stocks_post,50000)
bert_topic_modeling(stocks_post,len(stocks_post['title']))

In [ ]:
topic_model.save("Stocks_Post_BERTopic_DTM_30")

In [ ]:
topic_model.visualize_barchart(top_n_topics=30)

### Investing

In [ ]:
investing_pre.head(1)

In [ ]:
investing_pre['title'] = investing_pre['title'].astype('str')

In [ ]:
%%capture
topic_model = BERTopic( verbose=True)
#bert_topic_modeling(investing_pre,50000)
bert_topic_modeling(investing_pre,len(investing_pre['title']))


In [ ]:
topic_model.save("investing_Pre_BERTopic_DTM_30")

In [ ]:
topic_model.get_topic_info()[0:50]

In [ ]:
topic_model.visualize_barchart(top_n_topics=30)

In [ ]:
investing_post['title'] = investing_post['title'].astype('str')

In [ ]:
len(investing_post['title'])

In [ ]:
%%capture
topic_model = BERTopic( verbose=True)
#bert_topic_modeling(investing_post,50000)
bert_topic_modeling(investing_post,len(investing_post['title']))

In [ ]:
topic_model.save("investing_Post_BERTopic_DTM_30")

In [ ]:
topic_model.visualize_barchart(top_n_topics=30)

## Common Topics

In [23]:
def common_elements(list1,list2,list3=[]):
    users_wsb_stocks = set(list1) & set(list2) 
    print('Elements in List 1:',len(list1))
    print('Elements in List 2:',len(list2))
    print('common_elements [List 1,2]:',len(users_wsb_stocks))
    #print('common_elements:',users_wsb_stocks)
    if list3:
      print('Elements in List 3:',len(list3))
      users_wsb_investing = set(list1) & set(list3)
      print('common_elements [List 1,3]:',len(users_wsb_investing))
      #print('common_elements:',users_wsb_investing)
      users_stocks_investing = set(list2) & set(list3)
      print('common_elements [List 2,3]:',len(users_stocks_investing))
      #print('common_elements:',users_stocks_investing)  
      users = set(list1) & set(list2) & set(list3)
      print('common_elements [List 1,2,3]:',len(users))
      #print('common_elements:',users)
      return list(users)

In [ ]:
topics_over_time = topic_model.topics_over_time(docs=titles[0:50000], 
                                                topics=topics, 
                                                timestamps=timestamps[0:50000], 
                                                global_tuning=True, 
                                                evolution_tuning=True, 
                                                nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [ ]:
## Function to get common topics across all subreddits
def get_common_topics(topics1,topics2,topics3={}, common_words_nr= 5):
  # get list of words for each topic 
  # intersect operation on list of words of two topics
  # if intersection has atlest 1 or 2 common words : topic is same
  # add it to list of common topics : dictionary - {topic1: [word1,word2], topic2: [word1,word2] }
  common_topics = {}
  count = 0
  print('Topics in 1:'+ str(len(topics1.items())))
  print('Topics in 2:'+ str(len(topics2.items())))
  if topics3:
      print('Topics in 3:'+ str(len(topics3.items())))
  for topic,word_lst1 in topics1.items():
    topic_words = list(map(lambda x: x[0], word_lst1))
    #print(topic_words)
   
    for topic2,word_lst2 in topics2.items():
      topic2_words =  list(map(lambda x: x[0], word_lst2))
      #print(topic2_words)
      if topics3:        
        for topic3,word_lst3 in topics3.items():
          topic3_words =  list(map(lambda x: x[0], word_lst3))
          common_words = set(topic_words) & set(topic2_words) & set(topic3_words)
          if len(list(common_words)) > common_words_nr:
              print(common_words)
              count = count + 1
              common_topics['total'] = count 
              common_topics[topic]= topic_words
              #common_topics[topic]= word_lst1
              continue
              continue
              continue
          
      else:
        common_words = set(topic_words) & set(topic2_words)
        #print(list(common_words))
        #print(len(list(common_words)))
        
        if len(list(common_words)) > common_words_nr:
              print(common_words)
              count = count + 1
              common_topics['total'] = count 
              common_topics[topic]= topic_words
              #common_topics[topic]= word_lst1
              continue
              continue
  return common_topics

In [ ]:
investing_post = BERTopic.load("/content/drive/MyDrive/investing/investing_Post_BERTopic_DTM_30")
investing_pre = BERTopic.load("/content/drive/MyDrive/investing/investing_Pre_BERTopic_DTM_30")

In [ ]:
wsb_post = BERTopic.load("/content/drive/MyDrive/WSB/WSB_Post_BERTopic_DTM_30")
wsb_pre = BERTopic.load("/content/drive/MyDrive/WSB/WSB_Pre_BERTopic_DTM_30")

In [ ]:
stocks_pre = BERTopic.load("/content/drive/MyDrive/stocks/Stocks_Pre_BERTopic_DTM_30")
stocks_post = BERTopic.load("/content/drive/MyDrive/stocks/Stocks_Post_BERTopic_DTM_30")

In [ ]:
print(get_common_topics(wsb_post.get_topics(),investing_post.get_topics()))

In [ ]:
print(get_common_topics(wsb_post.get_topics(),wsb_pre.get_topics()))

In [ ]:
print(get_common_topics(investing_pre.get_topics(),investing_post.get_topics()))

In [ ]:
print(get_common_topics(stocks_pre.get_topics(),stocks_post.get_topics()))

In [ ]:
print(get_common_topics(wsb_pre.get_topics(),investing_pre.get_topics(),stocks_pre.get_topics()))

In [ ]:
print(get_common_topics(wsb_post.get_topics(),investing_post.get_topics(),stocks_post.get_topics()))

## Embeddings

## Analyse how contexts of a word change over time (e.g. temporal word embeddings, HistWords)

In [ ]:
#sentence, spacy embedding of titles: pre vs post

## If so, which changes are observable? (usage of emojis, choice of words)

In [ ]:
# top emojis, words: pre vs post

## Are new users using old WSB patterns inflationary?

In [11]:
stocks = pd.read_csv('stocks/data/stocks_submissions_all.csv',sep=';')


In [14]:
investing= pd.read_csv('investing/data/investing_submissions_all.csv',sep=';')

C:\Users\96ank\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (1,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
common_users= {'folkwoodswest', 'bearsgotoalaskanstfu', 'learner4f', 'ilikelucy1', 'CIARRAPUNGI', 'automax', 'hhh888hhhh', 'Hard_working247', 'WhichEdge', 'slow_down_more',
                  'hello-world-foo-bar', 'stillness0072', 'Slim-JayS', 'Griffin90', 'QuadraDGoesReddit', 'TheStonksHub', 'MountainsMan55', 'KingTimKap', 'BenDoverR8Now'}

In [ ]:
# overall top common users
common_users= {'moazzam0', 'ogordained', 'nafizzaki', 'seebz69', 'Michael12390', 'rawrtherapybackup'}

#for user in list(common_users):
wsb_[wsb_['author'] =='automax']['title']

In [ ]:
wsb_[wsb_['author'] =='folkwoodswest']['title'].tolist()

In [ ]:
stocks[stocks['author'] =='folkwoodswest']['title'].tolist()

In [ ]:
investing[investing['author'] =='folkwoodswest']['title'].tolist()

In [15]:
## create datasets of common users
wsb_users = wsb_['author'].unique().tolist()
stocks_users = stocks['author'].unique().tolist()
investing_users = investing['author'].unique().tolist()

In [16]:
len(wsb_users)

396127

In [17]:
len(stocks_users)

84609

In [18]:
len(investing_users)

73952

In [24]:
common_users = common_elements(wsb_users,stocks_users,investing_users)

Elements in List 1: 396127
Elements in List 2: 84609
common_elements [List 1,2]: 21271
Elements in List 3: 73952
common_elements [List 1,3]: 16413
common_elements [List 2,3]: 13964
common_elements [List 1,2,3]: 5637


In [26]:

wsb_common_users_data = wsb_[wsb_['author'].isin(common_users)]
stocks_common_users_data = stocks[stocks['author'].isin(common_users)]
investing_common_users_data = investing[investing['author'].isin(common_users)]

In [27]:
wsb_common_users_data.head()

,Unnamed: 0,created_utc,SegmentID,author,id,link_flair_text,num_comments,score,subreddit_subscribers,selftext,title,upvote_ratio,embedding,bigram,trigram,four-gram,five-gram,year,date
9,9,2021-10-19 17:46:46,9,Sciencetist,qbgrz6,DD,1,1,10967058,"What up, nerds? Long time no post. I come bear...",Don't Be A Menace To Hedge Funds By Squeezing ...,1.0,"[0.13588201999664307, -0.32027190923690796, -0...","[Hedge Funds, $HOOD]","[Menace To Hedge, Funds By Squeezing, Squeezin...","[Menace To Hedge Funds, Hedge Funds By Squeezi...","[Funds By Squeezing Their Shorts, Squeezing Th...",2021,2021-10-19
16,16,2021-10-19 17:41:55,16,Lost-Guarantee229,qbgo6x,DD,0,1,10967049,"As many of you know, [$TSLA - Tesla](https://u...","TSLA Earnings Prediction: They beat estimates,...",1.0,"[-0.3805695176124573, -0.3852122128009796, -0....","[TSLA Earnings, Earnings Prediction, beat esti...",[TSLA Earnings Prediction],[],[],2021,2021-10-19
24,24,2021-10-19 17:35:43,24,Desperate-invest-56,qbgja6,Gain,0,1,10967030,NaN,$ADXS continue to buy more and gain more,1.0,"[-0.020925845950841904, 0.01783006079494953, 0...","[$ADXS, ADXS continue]","[$ADXS continue, continue to buy]","[ADXS continue to buy, buy more and gain]",[$ADXS continue to buy],2021,2021-10-19
69,69,2021-10-19 16:42:31,69,el_gato14,qbffb6,Discussion,0,1,10966837,Do you guys play his call outs too? I mean I c...,Do you guys follow zack Morris also?,1.0,"[0.1666419506072998, -0.5782673954963684, -0.0...","[guys follow, follow zack, zack Morris]","[guys follow zack, follow zack Morris]",[guys follow zack Morris],[],2021,2021-10-19
74,74,2021-10-19 16:38:40,74,Postoaster,qbfcfl,Loss,0,1,10966829,NaN,What saith thou of my portfolio’th,1.0,"[0.02445914037525654, 0.19092664122581482, -0....",[saith thou],[],[thou of my portfolio’th],[saith thou of my portfolio’th],2021,2021-10-19


In [28]:
wsb_common_users_data.to_csv("data/wsb_common_users_data.csv",sep=';',index=False)

In [29]:
stocks_common_users_data.to_csv("data/stocks_common_users_data.csv",sep=';',index=False)

In [30]:
investing_common_users_data.to_csv("data/investing_common_users_data.csv",sep=';',index=False)